In [1]:
import os

try:
    has_changed_dir
except:
    has_changed_dir = False

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

if IN_COLAB:
    %pip install datasets
    %pip install translate-toolkit
    #%pip install bitsandbytes

    !git clone https://github.com/MartinKirkegaardDK/KDS_MI.git

    if not has_changed_dir:
        os.chdir('KDS_MI')
        has_changed_dir = True
else:
    if not has_changed_dir:
        os.chdir('.')
        has_changed_dir = True

Running as a Colab notebook
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.4/745.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 7.9 MB/s eta 0:00:00
Cloning into 'KDS_MI'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 40 (delta 13), reused 28 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 277.31 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
%load_ext autoreload
%autoreload 2

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch import nn
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset


from classification_probes import TextClassificationDataset, ProbeTrainer, HookManager, ClassificationProbe

In [3]:
model_name = "roneneldan/TinyStories-1M"
model_name = "EleutherAI/pythia-14m"
model_name = "AI-Sweden-Models/gpt-sw3-356m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

try:
    hidden_size = model.config.n_embd
except AttributeError:
    hidden_size = model.config.hidden_size

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/955 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [4]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64000, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): GELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=64000, bias=False)
)

In [5]:
lab_map = {
    'da': 0,
    'en': 1,
    'is': 2,
    'nb': 3,
    'sv': 4
}

data_loc = 'data/antibiotic/'
ds = TextClassificationDataset.from_txt(data_loc + 'da.txt', lab_map['da'])
ds.add_from_txt(data_loc + 'en.txt', lab_map['en'])
ds.add_from_txt(data_loc + 'is.txt', lab_map['is'])
ds.add_from_txt(data_loc + 'nb.txt', lab_map['nb'])
ds.add_from_txt(data_loc + 'sv.txt', lab_map['sv'])

loader = DataLoader(ds, batch_size=32, shuffle=True)

In [6]:
# making a dataset that can hold the activations and labels.

class ActivationDataset(Dataset):

    def __init__(self):
        self.acts = []
        self.labels = []

    def add_with_mask(self, acts, labels, masks):
        for act, label, mask in zip(acts, labels, masks):
            if mask:
                self.acts.append(act)
                self.labels.append(label)

    def __getitem__(self, index) -> tuple:
        return (self.acts[index], self.labels[index])

    def __len__(self) -> int:
        return len(self.acts)


In [7]:
activation_ds = ActivationDataset()

for text, label in loader:

    tokenized = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors='pt'
    ).to(device)

    with HookManager(model) as hook_manager:
        res_stream_act = hook_manager.attach_residstream_hook(
            layer=20,
            pre_mlp=False,
            pythia=False
        )

        model(**tokenized)

    # flattening [batch, pad_size, ...] to [tokens, ...]
    attn_mask = tokenized.attention_mask.flatten() # [tokens]
    label = label.unsqueeze(-1).expand(-1, tokenized.attention_mask.shape[1]).flatten() # [tokens]
    res_stream_act = res_stream_act[0].view(-1, hidden_size) # [tokens, hidden_size]

    activation_ds.add_with_mask(res_stream_act, label, attn_mask)



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
act_loader = DataLoader(activation_ds, batch_size=32, shuffle=True)

In [9]:
input_size = hidden_size
learning_rate = 0.001
reg_lambda = 0.1
num_labs = len(lab_map)

probe = ClassificationProbe(in_dim=input_size, num_labs=num_labs, device=device)
optimizer = torch.optim.Adam(probe.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    for act, label in act_loader:

        label = label.to(device)

        outputs = probe(act)
        loss = loss_fn(outputs, label.to(device))
        loss += reg_lambda * sum(torch.norm(param, 2) for param in probe.parameters())

        accuracy = ((torch.argmax(outputs.detach(), dim=1) == label.to(device)).sum() / 32).item()
        print('acc: ', accuracy, end='\t\t')

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(loss)

Streaming af output blev afkortet til de sidste 5000 linjer.
acc:  1.0		tensor(1.0395, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.96875		tensor(1.0644, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.875		tensor(1.1508, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.96875		tensor(1.0922, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.90625		tensor(1.1700, device='cuda:0', grad_fn=<AddBackward0>)
acc:  1.0		tensor(1.0795, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.96875		tensor(1.1028, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.96875		tensor(1.0791, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.96875		tensor(1.0545, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.9375		tensor(1.1444, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.96875		tensor(1.0841, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.96875		tensor(1.1046, device='cuda:0', grad_fn=<AddBackward0>)
acc:  0.9375		tensor(1.1117, device='cuda:0', grad_fn=<AddBackward0>)
acc:  1.0		tensor(1.1423, de

KeyboardInterrupt: 

In [30]:
neg_steering_vectors

[tensor([-0.0174, -0.0295,  0.0189,  ...,  0.0031,  0.0179,  0.0003],
        device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([-0.0054, -0.0103, -0.0040,  ..., -0.0178, -0.0134,  0.0039],
        device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([ 0.0166,  0.0164, -0.0061,  ..., -0.0125,  0.0593, -0.0018],
        device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([ 0.0200,  0.0101,  0.0010,  ...,  0.0082,  0.0157, -0.0021],
        device='cuda:0', grad_fn=<SelectBackward0>)]

In [13]:
steer_towards = 'da'

steering_vector = probe.linear._parameters['weight'][lab_map[steer_towards]]

neg_steering_vectors = [
    probe.linear._parameters['weight'][lab_map[lang]]
    for lang in lab_map
        if lang != steer_towards
]

text = 'This is a sentence, which'
tokenized = tokenizer(text, return_tensors='pt').to(device)

with HookManager(model) as hook_manager:
    hook_manager.attach_resid_stream_steer_hook(
        20,
        steering_vector,
        10,
        pre_mlp=False,
        pythia=False
    )

    for neg_steering_vector in neg_steering_vectors:
        hook_manager.attach_resid_stream_steer_hook(
            20,
            neg_steering_vector,
            -2,
            pre_mlp=False,
            pythia=False
        )

    output_da_steering = [
        model.generate(tokenized.input_ids, max_length=100, temperature=0.7, top_p=0.9, do_sample=True)
        for _ in range(10)
    ]

for output in output_da_steering:
    print(tokenizer.decode(output[0]).replace('\n', ' '))
    print()

This is a sentence, which is 1,100 words long, and the first sentence contains the phrase "I am a man".  Input: Sentence1: The main character is a man who is a doctor. Sentence2: He is a good doctor. Output: The main character is a doctor.  Input: Sentence1: The main character is a man who is a doctor. Sentence2: He is a good doctor. Output:

This is a sentence, which is not a sentence.  A man is standing on a table with his hands on his hips.  Input: Sentence 1: A man is standing on a table with his hands on his hips.  Output: A man is standing on a table with his hands on his hips.  Sentence 2: A man is standing on a table with his hands on his hips. Output: A man is standing on a table with

This is a sentence, which is the same as a sentence with one or more of the following structure: (1) (2) (3) (4) (5) (6) (7) (8) (9) (10) (11) (12) (13) (14) (15) (16) (17) (18) (19) (20) (21) (22)

 This is a sentence, which is not a sentence.  Question: Is the sentence a sentence or a sentence

In [ ]:
steering_vector = probe.linear._parameters['weight'][3]

text = 'Dette er'
tokenized = tokenizer(text, return_tensors='pt').to(device)

with HookManager(model) as hook_manager:
    hook_manager.attach_resid_stream_steer_hook(
        17,
        steering_vector,
        50,
        pre_mlp=False,
        pythia=False
    )

    output_nb_steering = [
        model.generate(tokenized.input_ids, max_length=100, temperature=0.7, top_p=0.9, do_sample=True)
        for _ in range(10)
    ]

for output in output_nb_steering:
    print(tokenizer.decode(output[0]).replace('\n', ' '))
    print()

Dette er ikke et krav, men en fordel.  6. 10.10.2016 11:58  Publisert: 10.10.2016 11:58  Publisert: 10.10.2016 11:58  Sist oppdatert:  P  P  P  P  P  P 

Dette er et godt eksempel på at du kan ha mange interesser, samtidig som du har en god plan for hvordan du skal bruke de. For å holde en god plan, er det viktig å sette av tid til å planlegge. Det er viktig at du får til å planlegge alt i detalj, både på tid, hvor du skal, hvor du skal ta det, hvilke tjenester du skal få, og hva du skal gjøre. Dette er et viktig punkt for å ha en plan, da det kan påvirke

Dette er en av de viktigste faktorene for å finne en løsning som er til beste for alle. I denne oppgaven vil jeg forsøke å ta en slik løsning til det den er til. Jeg ønsker å ta en god løsning på et problem som er til det beste for alle. Jeg ønsker å finne en løsning som er til det beste for de som trenger det. Jeg vil se på et problem som er til det beste for alle, og jeg ønsker å se på et problem som er til det beste

Dette er ikke

In [ ]:
output = model.generate(tokenized.input_ids, max_length=100, temperature=0.7, top_p=0.9, do_sample=False)

tokenizer.decode(output[0])

c:\Users\Bruger\Code\mechinterp\mechinterp\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Bruger\Code\mechinterp\mechinterp\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Arranged in six parts, the film depicts a strike in 1903 by the workers of a factory in pre-revolutionary Russia, and their subsequent suppression. It is best known for a sequence towards the climax, in which the violent and violent of the Soviet Union was the first to be seen.\n\nThe film is a film that is a film that is a film that is a film that is a film that is a film that is a film that is a film that is a film that'

In [ ]:
output

tensor([[ 7047,  3767, 30912,    75,   413,  2827,   546,  1448,  6706,   546,
          2098, 14293,    85,  1151,   466,   891,   726,    87, 33578,  1129,
          8225,  5025,   346,    49,  2768,  1670,    75,   771,   354,  1059,
         33325,  1507,  1073,  5751,  4953,     3,  9040,   945,  1162,   269,
          5507,   620,   265,  1775,   274,  1257,    75,   615,   278,  4415,
            78,    15,   187,   187,    34,    27,   187,   187,   510,   806,
          3213,   310,   281,  1918,   368,   247,  2372,   273,   247, 12662,
           285,   923,   752,   253,  3662,    27,   187,   187,    14, 50275,
           187,   187,    14, 50275,   187,   187,    14, 50275,   187,   187,
            14, 50275,   187,   187,    14, 50275,   187,   187,    14, 50275]])

'Filmen Strejke er en del af en planlagt serie i syv dele med titlen "På vej mod proletariatets diktatur" og var et fælles samarbejde mellem.\n\nA:\n\nThe first step is to give you a bit of a hint and see what the answer:\n\n-   \n\n-   \n\n-   \n\n-   \n\n-   \n\n-   '

In [ ]:
projection_magnitudes = (act.unsqueeze(0) @ steering_vector).unsqueeze(-1)

In [ ]:
steering_vector_ = steering_vector.view(1, 1, -1)

In [ ]:
projections

tensor([[[ 1.0383],
         [ 3.2407],
         [ 4.7199],
         [-0.4780],
         [ 4.3807],
         [-3.7402],
         [ 1.0187],
         [ 8.1848],
         [-1.3870],
         [ 1.7934],
         [ 0.7859],
         [-3.5573],
         [-0.1791],
         [ 2.3834]]], grad_fn=<UnsqueezeBackward0>)

In [ ]:
projections = (projection_magnitudes * steering_vector_)

tensor([4, 0, 3, 1, 4, 1, 2, 2, 4, 0, 2, 2, 1, 4, 4, 1, 4, 0, 4, 4, 1, 4, 0, 1,
        1, 4, 2, 2, 0, 3, 2, 1])

In [ ]:
label.unsqueeze(-1).expand(-1, tokenized.attention_mask.shape[1]).flatten()

tensor([2, 2, 2,  ..., 4, 4, 4])

In [ ]:
tokenized.attention_mask.flatten().shape

torch.Size([4928])

In [ ]:
res_stream_act[0].view(-1, hidden_size)[0]

tensor([-0.5304,  0.9247, -0.0046, -2.0886,  0.9416, -0.6766,  1.6014,  0.4292,
        -0.3467,  0.6638, -0.3799, -3.7725,  0.0168, -0.6235,  1.7516,  1.1082,
         0.2715,  0.7788,  0.1381,  0.3071,  1.7841,  0.8833, -0.9052,  0.7601,
        -1.4813,  0.8397, -2.0588, -0.9006, -0.7124,  0.6576, -1.1364,  1.0030,
         0.6364,  1.6032,  0.4388, -1.4383,  0.8074,  3.0726,  0.0916, -0.9084,
         0.0775,  1.0104, -0.6060, -0.0836, -2.3467, -4.3569,  2.1628,  1.2648,
        -0.7746, -0.0214,  1.1413, -0.5092, -2.0130,  0.4083, -2.4431, -1.1179,
         0.5212,  0.7194, -0.6159, -0.8679,  0.7901,  1.3831,  0.0591,  0.8055])

In [ ]:
res_stream_act[0].shape

torch.Size([32, 154, 64])

In [ ]:

trainer = ProbeTrainer(hidden_size, 5, 0.001, 0.1, 'cpu')

for text_batch, labels in loader:
    print(text)
    with HookManager(model) as hook_manager:
        res_stream_act = hook_manager.attach_residstream_hook(
            layer=4,
            pre_mlp=False
        )

        tokenized = [
            tokenizer(text, return_tensors='pt')
            for text in text_batch
        ]
        for text in tokenized:
            model.forward(**text)

    loss = trainer.train_step(torch.Tensor(res_stream_act), torch.Tensor(labels))
    print(loss)

In [ ]:
model.config.n_embed

AttributeError: 'GPTNeoConfig' object has no attribute 'n_embed'

In [ ]:
model.get_submodule('transformer.h.')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64000, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): GELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=64000, bias=False)
)

In [ ]:
input_ = "dette er en"

tokenized = tokenizer(input_, return_tensors='pt')

In [ ]:
tokenized

{'input_ids': tensor([[1122,  358,  315]]), 'attention_mask': tensor([[1, 1, 1]])}

In [ ]:
output_ = model.generate(tokenized.input_ids, max_length=100, temperature=0.7, top_p=0.9, do_sample=True)